In [8]:
import pandas as pd
import sys
import recordlinkage
from recordlinkage import preprocessing
from recordlinkage.standardise import clean
from sklearn.model_selection import cross_val_score
from sklearn import svm, model_selection, preprocessing
from recordlinkage.index import Full
import sklearn
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [12]:
 
 # Read the first file into pandas dataframe
 df1=pd.read_csv('bikedekho.csv')
 print(df1.shape) # total number of records in the file
 print(df1.head()) # to look at the first few rows
 print(df1.dtypes) # print the datatypes of all columns in the file
 print(df1['bike_name'].nunique())
 df1['bike_name']=clean(df1['bike_name']) #clean the string values in the bikename column

(4786, 10)
      id                               bike_name city_posted  km_driven  \
0  36082  Royal Enfield Bullet Electra Twinspark       Delhi        900   
1  36115           Royal Enfield Thunderbird 350      Mumbai      15000   
2  36118                Royal Enfield Bullet 350      Mumbai      50000   
3  36119                            KTM Duke 200      Mumbai      13000   
4  36120              Yamaha YZF R15 Version 2.0      Mumbai      28000   

   color fuel_type   price  model_year  owner_type                        url  
0  black    Petrol  110000        2015  FirstOwner  http://www.bikedekho.com/  
1  black    Petrol  110000        2013  FirstOwner  http://www.bikedekho.com/  
2   grey    Petrol  110000        1984  FifthOwner  http://www.bikedekho.com/  
3  white    Petrol  100000        2014  FirstOwner  http://www.bikedekho.com/  
4    red    Petrol   70000        2013  FirstOwner  http://www.bikedekho.com/  
id              int64
bike_name      object
city_posted   

In [14]:
# Now to look at the second file, read it into dataframe 2
df2=pd.read_csv('bikewale.csv')
print(df2.shape)   # get the total number of rows
print(df2.head())  # print first few rows to get an idea of how our data looks like
print(df2.dtypes)  # print the datatypes of each column
print(df2['bike_name'].nunique())  # lets look at the number of unique values in the bike name column
df2['bike_name']=clean(df2['bike_name']) # clean the string values in the bike name column using the "clean" library

(9003, 10)
   id                            bike_name city_posted  km_driven  color  \
0  12   TVS Apache RTR 160 Rear Drum Brake      Mumbai      28500   grey   
1  20              Bajaj Discover 125 Disc      Mumbai      21300  green   
2  26              Bajaj Pulsar 150 DTS- i   Bangalore      20000  black   
3  29  Royal Enfield Thunderbird Twinspark      Mumbai      20000  black   
4  53                       Hero Karizma R      Mumbai      35000  black   

  fuel_type  price  model_year owner_type  \
0    Petrol  40000        2010      First   
1    Petrol  27000        2009      First   
2    Petrol  70000        2010      First   
3    Petrol  85000        2008      First   
4    Petrol  45000        2007      First   

                                                 url  
0  http://www.bikewale.com/used/bikes-in-mumbai/t...  
1  http://www.bikewale.com/used/bikes-in-mumbai/b...  
2  http://www.bikewale.com/used/bikes-in-bangalor...  
3  http://www.bikewale.com/used/bikes-in-

In [18]:
# Until  now we have two files that we are going to link using the supervised classificatioin models. For that we also need
# labled data set. Lets look at the true matches/links that we have:

df1=pd.read_csv('labeled_data.csv',skiprows=5,usecols=[1,2])  # I am skipping first five rows just because there is glitch in
#formatting of this file

df1.columns= ['ltable.id','rtable.id']
print(df1.shape)
print(df1.head())

#This file contains the id's of the true match pairs from the bikedekho.csv and bikewale.csv file. To see how many unique
#records are there from each file since its possible that a record from FILE-1 has multiple links in the FILE-2 and vice versa

print(df1['ltable.id'].nunique()) 
print(df1['rtable.id'].nunique())

# Now we have id's of records from  both files , we need to get rest of the info for these records from their respective files

df2=pd.read_csv('bikedekho.csv')
# merge them on the ID's to get the rest of the columns from the df2
df4=pd.merge(df1,df2,left_on=['ltable.id'],right_on=['id'])
print(df4.shape) # print the number of records to make sure we are doing it right
print(df4.head()) # have a look at first few rows
print(df4.columns) # print the column names to make sure we are getting all of the columns

# Writing the true links into a separate file    
df4.to_csv('bikedekho_lable.csv',index=False)
    
# Repeat the same steps for 'bikewale.csv' file
df3=pd.read_csv('bikewale.csv')
df5=pd.merge(df1,df3,left_on=['rtable.id'],right_on=['id'])
print(df5.shape)
print(df5.head())
print(df5.columns)
df5.to_csv('bikewale_lable.csv',index=False)


(445, 2)
   ltable.id  rtable.id
0      36442       1820
1      36480      23151
2      36480      27834
3      36717      33756
4      36734      11034
400
421
(445, 12)
   ltable.id  rtable.id     id               bike_name city_posted  km_driven  \
0      36442       1820  36442        Bajaj Pulsar 135       Delhi      17000   
1      36480      23151  36480  Hero Honda Passion Pro       Delhi      10200   
2      36480      27834  36480  Hero Honda Passion Pro       Delhi      10200   
3      36717      33756  36717         Hero Honda Hunk       Delhi      33000   
4      36734      11034  36734          Yamaha YZF R15       Delhi      17000   

   color fuel_type  price  model_year   owner_type                        url  
0    red    Petrol  27999        2010  SecondOwner  http://www.bikedekho.com/  
1  black    Petrol  32000        2013   FirstOwner  http://www.bikedekho.com/  
2  black    Petrol  32000        2013   FirstOwner  http://www.bikedekho.com/  
3    red    Petrol  27

In [21]:
# now that we have these true matches into their own consecutive files, next step is to apply full indexing to get all possible pairs and comparing them using 
# different comparison methods to generate the similarity scores between them.

df1=pd.read_csv('bikedekho_lable.csv',sep=',', usecols=[2,3,4,5,6,7,8,9,10,11])  # only using the columns that were used to generate the labeled data
df1.set_index('id', inplace=True) # set id as index
df1 = df1[~df1.index.duplicated(keep='first')] # remove any duplicate values 


# repeating the above three steps for the second file as well    
df2=pd.read_csv('bikewale_lable.csv', sep=',', usecols=[2,3,4,5,6,7,8,9,10,11])
df2.set_index('id', inplace=True)
df2 = df2[~df2.index.duplicated(keep='first')]

# A full index is an index with all possible combinations of record pairs. In case of linking, this indexation method generates the cartesian product of both DataFrame’s. 
indexer=recordlinkage.index.Full()
pcl=indexer.index(df1,df2)
print(pcl.shape)

# We can also divide the pairs into chunks if we have some sort of memory issue, so that the processing will be done without any error

x = recordlinkage.index_split(pcl, 1000)
for chunk in x:
    
# for each chunk of 1000 we compare the attributes/features of the pairs to compute the similarity score between them.

    compare_cl = recordlinkage.Compare()   

# to compare the string type we are using the jarowinkler similarity. 

    compare_cl.string('bike_name', 'bike_name', method='jarowinkler', label='bikenameJW')
    compare_cl.string('city_posted', 'city_posted', method='jarowinkler', label='citypostedJW')
    compare_cl.string('color', 'color', method='jarowinkler', label='colorJW')
    compare_cl.string('fuel_type', 'fuel_type', method='jarowinkler', label='fueltypeJW')
        
# to compare the integer type we are using the exact comparison where the function returns the value 1 if the values are totally similar otherwise it returns the value 0.
    compare_cl.exact('km_driven', 'km_driven', label='km_driven')
    compare_cl.exact('price', 'price', label='price')
    compare_cl.exact('model_year', 'model_year', label='model_year')
    
    features = compare_cl.compute(chunk, df1, df2)  # Compare the records of each record pair. Calling this method starts the comparing of records.
       
    with open('labelFeature.csv', 'a') as f:    # the feature vectors computed for the pairs are written into this file.
            features.to_csv(f, header=False)

Index(['bike_name', 'city_posted', 'km_driven', 'color', 'fuel_type', 'price',
       'model_year', 'owner_type', 'url'],
      dtype='object')
(421, 9)
(168400,)


In [23]:
df=pd.read_csv('labelFeature.csv')
df1=pd.read_csv('labeled_data.csv',usecols=[1,2,13])
print(df1.head())
# getting the matches and non-matchces into separate files to extract their features from df
df2=df1[df1['gold']==1]
print(df2.shape)
df3=df1[df1['gold']==0]
print(df3.shape)
df4=pd.merge(df,df2,left_on=['lID','rID'],right_on=['ltable.id','rtable.id'])
print(df4.shape)
df5=pd.merge(df,df3,left_on=['lID','rID'],right_on=['ltable.id','rtable.id'])
print(df5.shape)
df6 = pd.concat([df4, df5])
print(df6.shape)
df6.to_csv('Finaltrain.csv', index=False)

   ltable.id  rtable.id  gold
0      36217      28620     0
1      36219      31966     0
2      36224      24610     1
3      36243      22741     0
4      36245      29302     1
(130, 3)
(320, 3)
(130, 13)
(319, 13)
(449, 13)


In [25]:
# Lets train our Random Forest and Support Vector Machine Classifiers
df1=pd.read_csv('Finaltrain.csv')
X=df1.drop('gold',axis=1)

y=df1['gold']

# dividing our train data into train and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
clf=RandomForestClassifier(n_estimators=100,max_features='sqrt',max_depth=200,min_samples_leaf=2,min_samples_split=6,bootstrap=False)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[[150  12]
 [  7  56]]
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       162
           1       0.82      0.89      0.85        63

    accuracy                           0.92       225
   macro avg       0.89      0.91      0.90       225
weighted avg       0.92      0.92      0.92       225

Accuracy: 0.9155555555555556
